# Simulation 2. Wahlgang Ständerat Aargau 2023

Eingelegte Wahlzettel 2019: 37,4 Prozent der Stimmberechtigten: https://www.ag.ch/app/wap/?years=2019#/20191124/29/srw

Wir rechnen mit den fiktiven Wählenden pro Partei. Auf diese Zahl kommt man, wenn man die gültigen Wahlzettel den Parteistärken entsprechend auf die Parteien aufteilt. Zusätzlich betrachten wir die Abstinenten. Diese definieren wir der Einfachheit halber inklusive leerer und ungültiger Wahlzettel. Diese machen im Aargau zusammen nur 0,73 Prozent der Wahlberechtigten aus.

Szenarien-Begründung:
- Von der FDP hat Giezendanner 4198 Panaschierstimmen erhalten, von Binder 3630. Der Anteil beträgt somit 53,6 zu 46,4 Prozent pro Giezendanner. Nimmt man die Empfehlung der FDP für Giezendanner hinzu, dürften sich die Kräfteverhältnisse zu Gunsten des SVP-Mannes verschieben. Wir nehmen darum im ersten Szenario eine Verteilung der FDP-Stimmen im Verhältnis 80:20 resp. 4:1 für Giezendanner an.
- Bei SP, Grünen und GLP sind die Verhältnisse fast identisch: Vom Panaschierstimmentopf für Giezendanner/Binder erhielt Binder fast 87 Prozent (SP, Grüne) resp. fast 84 Prozent (GLP) der Stimmen. Mit den Empfehlungen der Parteien für Binder nehmen wir darum an, dass bei SP und Grünen alle Stimmen an Binder gehen, bei der GLP alle ausser ein Prozent.
- Bei der EVP betrug das Panaschierverhältnis zwischen Giezendanner und Binder 34,2:65,8, also ziemlich genau 2:3. Weil die EVP Binder empfiehlt, dürfte sich das Verhältnis noch akzentuieren. Wir gehen von einem Verhältnis von 5:1 zugunsten Binders aus.
- Bei den übrigen Parteien macht die EDU den grössten Teil aus. Aber auch die Aussenseiter-Kandidaturen Lischer und Holten machen insgesamt einige Stimmen. Der Einfachheit halber weisen wir ihnen Stimmen ausschliesslich von den übrigen Parteien zu. Darüber hinaus dürfte Giezendanner bei den übrigen - wegen der EDU - stärker punkten als Binder, weshalb wir von einem Verhältnis von 5:1 für Giezendanner ausgehen.
- Weiter gehen wir davon aus, dass de Abstinenten der Nationalratswahl sich auch am 19. November nicht beteiligen werden. Die Wahlbeteiligung ist in zweiten Wahlgängen in der Regel tiefer als in den ersten. Ausserdem fehlen die Anhaltspunkte, um diese Wählergruppe den Parteien zuzuordnen.

In [1]:
import requests
import pandas as pd
import numpy as np
from my_wahl_settings import parteien_zuordner

In [2]:
df_partei = pd.read_csv(parteien_zuordner)

In [3]:
df_partei

,partei_id,partei_kurz,logo
0,1,FDP,https://chm-editorial-data-static.s3.eu-west-1...
1,2,CVP,NaN
2,3,SP,https://chm-editorial-data-static.s3.eu-west-1...
3,4,SVP,https://chm-editorial-data-static.s3.eu-west-1...
4,5,LPS,NaN
5,6,LdU,NaN
6,7,EVP,https://chm-editorial-data-static.s3.eu-west-1...
7,8,CSP,NaN
8,9,PdA/Sol.,NaN
9,12,FGA,NaN


Import Wahlberechtigte

In [4]:
url_be = 'https://ogd-static.voteinfo-app.ch/v4/ogd/sd-t-17.02-NRW2023-wahlbeteiligung.json'

r_be = requests.get(url_be)

r_be = r_be.json()

df_berechtigte = pd.DataFrame(r_be['level_kantone'])

gueltige_wahlzettel = df_berechtigte[df_berechtigte['kanton_nummer'] == 19]['gueltige_wahlzettel'].values[0]

wahlberechtigte = df_berechtigte[df_berechtigte['kanton_nummer'] == 19]['wahlberechtigte'].values[0]

Wie gross ist der Anteil leerer und ungültiger Wahlzettel?

In [5]:
df_berechtigte['leer_ungültig_pct'] = (df_berechtigte['leere_wahlzettel'] + df_berechtigte['ungueltige_wahlzettel']) / df_berechtigte['wahlberechtigte'] * 100

Import Wahldaten

In [6]:
url = 'https://ogd-static.voteinfo-app.ch/v4/ogd/sd-t-17.02-NRW2023-parteien.json'

r = requests.get(url)

r = r.json()

In [7]:
df = pd.DataFrame(r['level_kantone'])

df = df[df['kanton_nummer'] == 19].copy()

df = df_partei[['partei_id', 'partei_kurz']].merge(df, on='partei_id', how='right')

df = df[['partei_kurz', 'fiktive_waehlende']].copy()

In [8]:
df

,partei_kurz,fiktive_waehlende
0,FDP,26492.0
1,SP,33328.0
2,SVP,72030.0
3,EVP,7025.0
4,PdA/Sol.,224.0
5,Grüne,14448.0
6,EDU,2010.0
7,GLP,17238.0
8,Mitte,24354.0
9,Übrige,5797.0


Abstinente dazu fügen (Nichtwähler plus leere plus ungültige)

In [9]:
fiktive_abwesende = wahlberechtigte - gueltige_wahlzettel

abstinenten_dict = {'partei_kurz': ['Abstinente'], 'fiktive_waehlende': [fiktive_abwesende]}

df = pd.concat([df, pd.DataFrame(abstinenten_dict)])

Kleinparteien zu Übrige

In [10]:
df.loc[df['partei_kurz'].isin(['PdA/Sol.', 'EDU']), 'partei_kurz'] = 'Übrige'

df = df.groupby('partei_kurz')[['fiktive_waehlende']].sum()

Kontrolle: Haben wir gleich viele fiktive Wählende wie Wahlberechtigte?

In [11]:
wahlberechtigte == df['fiktive_waehlende'].sum()

True

Szenario-Daten hinzufügen. Es handelt sich um Prozentwerte, entlang denen die fiktiven Wählenden pro Partei an die Kandidaten verteilt werden. Es müssen immer 100 Prozent sein.

In [12]:
df_szenario = pd.read_excel('szenario.xlsx', index_col='Partei')

df = df.join(df_szenario)

In [13]:
len(df_szenario.sum(axis=1)[df_szenario.sum(axis=1) == 100]) == len(df_szenario)

True

In [14]:
df

,fiktive_waehlende,Giezendanner,Binder,Übrige,Enthaltung
partei_kurz,,,,,
Abstinente,239935.0,0,0,0,100
EVP,7025.0,10,50,0,40
FDP,26492.0,48,12,0,40
GLP,17238.0,1,59,0,40
Grüne,14448.0,0,60,0,40
Mitte,24354.0,0,95,0,5
SP,33328.0,0,60,0,40
SVP,72030.0,95,0,0,5
Übrige,8031.0,25,5,30,40


**Funktion zur Berechnung der Wähleranteile**

In [15]:
def ergebnis_rechner(df):
        
    df_func = df.copy()
    
    #Wir berechnen die Wähler pro Kandidat/in
    df_func['wähler_giezendanner'] = df_func['fiktive_waehlende'] * (df_func['Giezendanner'] / 100)
    df_func['wähler_binder'] = df_func['fiktive_waehlende'] * (df_func['Binder'] / 100)
    df_func['wähler_übrige'] = df_func['fiktive_waehlende'] * (df_func['Übrige'] / 100)
    df_func['wähler_enthaltung'] = df_func['fiktive_waehlende'] * (df_func['Enthaltung'] / 100)

    df_func = df_func.replace(np.inf, 0)
    
    first_check = wahlberechtigte == round(df_func['wähler_giezendanner'].sum() + df_func['wähler_binder'].sum() + df_func['wähler_übrige'].sum() + df_func['wähler_enthaltung'].sum(),4)
       
    if first_check != True:
        raise ValueError('Mit der Wählerzuteilung stimmt etwas nicht.')
    
    summe_giezendanner_binder = df_func['wähler_giezendanner'].sum() + df_func['wähler_binder'].sum() + df_func['wähler_übrige'].sum()
    
    #Wähleranteile
    wähleranteil_giezendanner = df_func['wähler_giezendanner'].sum() / summe_giezendanner_binder * 100
    wähleranteil_binder = df_func['wähler_binder'].sum() / summe_giezendanner_binder * 100
    wähleranteil_übrige = df_func['wähler_übrige'].sum() / summe_giezendanner_binder * 100
    
    wähler_giezendanner = df_func['wähler_giezendanner'].sum()
    wähler_binder = df_func['wähler_binder'].sum()
    wähler_übrige = df_func['wähler_übrige'].sum()
    
    kandi_check = wähleranteil_giezendanner + wähleranteil_binder + wähleranteil_übrige
    
    if kandi_check < 99.9 or kandi_check > 100.1:
        raise ValueError('Irgendetwas stimmt mit den Wähleranteilen von Giezendanner oder Binder nicht.')
        
    #Enthaltung / Beteiligung
    anteil_enthaltung = df_func['wähler_enthaltung'].sum() / wahlberechtigte * 100
    
    wahlbeteiligung = 100 - anteil_enthaltung
    
    if wahlbeteiligung > 100 or wahlbeteiligung < 10:
        raise ValueError('Irgendetwas stimmt mit der Wahlbeteiligung nicht.')
        
    result_dict = {
        'wähleranteil_giezendanner': [wähleranteil_giezendanner],
        'wähleranteil_binder': [wähleranteil_binder],
        'wähleranteil_übrige': [wähleranteil_übrige],
        'wähler_giezendanner': [wähler_giezendanner],
        'wähler_binder': [wähler_binder],
        'wähler_übrige': [wähler_übrige],
        'anteil_enthaltung': [anteil_enthaltung],
        'wahlbeteiligung': [wahlbeteiligung]
    }
    
    df_results = pd.DataFrame(result_dict)
    
    return df_results

**Berechnungen - Szenario 0**

Ausgangslage: SP, Grüne und GLP unterstützen praktisch geschlossen Binder, die FDP-Wählenden gehen im Verhältnis 4:1 an Giezendanner. Von SVP und Mitte nehmen fünf Prozent der Wählenden nicht an der Wahl teil, von den übrigen Parteien 40 Prozent.

In [16]:
ergebnis_rechner(df)

,wähleranteil_giezendanner,wähleranteil_binder,wähleranteil_übrige,wähler_giezendanner,wähler_binder,wähler_übrige,anteil_enthaltung,wahlbeteiligung
0,54.036147,44.414483,1.549369,84027.29,69065.41,2409.3,64.888537,35.111463


In [17]:
round(84027.29)

84027

**Berechnungen - Szenario 1**

Ausgangsfrage: Wir prüfen, wie stark sich die linken Wähler (einmal inklusive GLP) beteiligen müssen, bis Binder Giezendanner überholt.

In [18]:
df_szenario1 = df.copy()

loop_binder1 = ergebnis_rechner(df_szenario1)['wähler_binder'].values[0]
loop_giezendanner1 = ergebnis_rechner(df_szenario1)['wähler_giezendanner'].values[0]

while loop_binder1 < loop_giezendanner1:
    df_szenario1.loc['SP', 'Binder'] += 1
    df_szenario1.loc['SP', 'Enthaltung'] -= 1
    
    df_szenario1.loc['Grüne', 'Binder'] += 1
    df_szenario1.loc['Grüne', 'Enthaltung'] -= 1

#Inklusive GLP
#     df_szenario1.loc['GLP', 'Binder'] += 1
#     df_szenario1.loc['GLP', 'Enthaltung'] -= 1
    
    if df_szenario1.loc['SP', 'Enthaltung'] < 0:
        raise ValueError('Enthaltung unter Null.')
    
    df_ergebnis_temp1 = ergebnis_rechner(df_szenario1)
    
    loop_binder1 = df_ergebnis_temp1.loc[0, 'wähler_binder']
    loop_giezendanner1 = df_ergebnis_temp1.loc[0, 'wähler_giezendanner']

In [19]:
df_ergebnis_temp1

,wähleranteil_giezendanner,wähleranteil_binder,wähleranteil_übrige,wähler_giezendanner,wähler_binder,wähler_übrige,anteil_enthaltung,wahlbeteiligung
0,49.199094,49.390229,1.410677,84027.29,84353.73,2409.3,61.436521,38.563479


In [20]:
df_szenario1

,fiktive_waehlende,Giezendanner,Binder,Übrige,Enthaltung
partei_kurz,,,,,
Abstinente,239935.0,0,0,0,100
EVP,7025.0,10,50,0,40
FDP,26492.0,48,12,0,40
GLP,17238.0,1,59,0,40
Grüne,14448.0,0,92,0,8
Mitte,24354.0,0,95,0,5
SP,33328.0,0,92,0,8
SVP,72030.0,95,0,0,5
Übrige,8031.0,25,5,30,40


**Berechnungen - Szenario 2**

Ausgangsfrage: Was passiert, wenn die FDP bis zur Hälfte Binder unterstützt?

In [21]:
df_szenario2 = df.copy()

loop_binder2 = ergebnis_rechner(df_szenario2)['wähler_binder'].values[0]
loop_giezendanner2 = ergebnis_rechner(df_szenario2)['wähler_giezendanner'].values[0]

while loop_binder2 < loop_giezendanner2:
    
    df_szenario2.loc['FDP', 'Binder'] += 1
    df_szenario2.loc['FDP', 'Giezendanner'] -= 1
    
    if df_szenario2.loc['FDP', 'Binder'] > 30:
        raise ValueError('Binder über 30 und damit über der Hälfte der FDP.')
    
    df_ergebnis_temp2 = ergebnis_rechner(df_szenario2)
    
    loop_binder2 = df_ergebnis_temp2.loc[0, 'wähler_binder']
    loop_giezendanner2 = df_ergebnis_temp2.loc[0, 'wähler_giezendanner']

ValueError: Binder über 30 und damit über der Hälfte der FDP.

In [22]:
df_ergebnis_temp2

,wähleranteil_giezendanner,wähleranteil_binder,wähleranteil_übrige,wähler_giezendanner,wähler_binder,wähler_übrige,anteil_enthaltung,wahlbeteiligung
0,50.969589,47.481042,1.549369,79258.73,73833.97,2409.3,64.888537,35.111463


In [23]:
df_szenario2

,fiktive_waehlende,Giezendanner,Binder,Übrige,Enthaltung
partei_kurz,,,,,
Abstinente,239935.0,0,0,0,100
EVP,7025.0,10,50,0,40
FDP,26492.0,29,31,0,40
GLP,17238.0,1,59,0,40
Grüne,14448.0,0,60,0,40
Mitte,24354.0,0,95,0,5
SP,33328.0,0,60,0,40
SVP,72030.0,95,0,0,5
Übrige,8031.0,25,5,30,40


**Berechnungen - Szenario 3**

Ausgangsfrage: Was passiert, wenn die Wahlbeteiligung bei allen Parteien hoch ist? Dazu lesen wir ein Extra-Szenario ein.

In [24]:
df_szenario3 = df.copy()

df_szenario_extra = pd.read_excel('szenario_10.xlsx')

df_szenario3 = df_szenario3[['fiktive_waehlende']].join(df_szenario_extra.set_index('Partei'))

df_ergebnis_temp3 = ergebnis_rechner(df_szenario3)

In [25]:
df_ergebnis_temp3

,wähleranteil_giezendanner,wähleranteil_binder,wähleranteil_übrige,wähler_giezendanner,wähler_binder,wähler_übrige,anteil_enthaltung,wahlbeteiligung
0,50.070545,48.670191,1.259265,95797.95,93118.71,2409.3,56.799691,43.200309


**Berechnungen - Szenario 4**

Ausgangsfrage: Was passiert, wenn Binder bei den Linken und der FDP abräumt?

In [26]:
df_szenario4 = df.copy()

df_szenario_extra4 = pd.read_excel('szenario_pro_binder.xlsx')

df_szenario4 = df_szenario4[['fiktive_waehlende']].join(df_szenario_extra4.set_index('Partei'))

df_ergebnis_temp4 = ergebnis_rechner(df_szenario4)

In [27]:
df_szenario4

,fiktive_waehlende,Giezendanner,Binder,Übrige,Enthaltung
partei_kurz,,,,,
Abstinente,239935.0,0,0,0,100
EVP,7025.0,10,50,0,40
FDP,26492.0,30,30,0,40
GLP,17238.0,1,79,0,20
Grüne,14448.0,0,80,0,20
Mitte,24354.0,0,95,0,5
SP,33328.0,0,80,0,20
SVP,72030.0,95,0,0,5
Übrige,8031.0,25,5,30,40


In [28]:
df_ergebnis_temp4

,wähleranteil_giezendanner,wähleranteil_binder,wähleranteil_übrige,wähler_giezendanner,wähler_binder,wähler_übrige,anteil_enthaltung,wahlbeteiligung
0,47.036482,51.533707,1.429811,79258.73,86836.77,2409.3,61.952579,38.047421
